In [9]:
import cv2
import numpy as np
import os
import cvlib as cv
from cvlib.object_detection import draw_bbox

In [159]:
import ipywidgets as widgets
from IPython.display import display

# Creates sliders 
minClipLength = widgets.IntSlider(
    value=5,
    min=2,
    max=200,
    step=1,
    description='Minimum clip length:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%') 
)

minComponentSize = widgets.IntSlider(
    value=100,
    min=10,
    max=800,
    step=5,
    description='Minimum component size:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%') 
)

trackingFrames = widgets.IntSlider(
    value=10,
    min=0,
    max=50,
    step=1,
    description='Number of frames to apply tracking:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%') 
)

trackingBboxScaleFactor = widgets.FloatSlider(
    value=.5,
    min=0.1,
    max=1,
    step=.1,
    description='Factor to resize tracking bounding box:',
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%') 
)

# Text input for video path or frame folder path
input_path = widgets.Text(
    value='',
    placeholder='Enter name',
    description='Input folder/file name:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Text input for output folder name
output_folder = widgets.Text(
    value='result',
    placeholder='Enter name',
    description='Output folder name:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

# Displays the widgets
widget_container = widgets.VBox([
    minClipLength, minComponentSize, trackingFrames, trackingBboxScaleFactor, 
    input_path, output_folder
])

display(widget_container)


In [26]:
# Keeps only large connected components in the frame
def keepLargeComponents(I,th):
    R = np.zeros(I.shape)<0
    unique_labels = np.unique(I)
    for label in unique_labels:
        if label == 0:
            pass
        else:
            I2 = I==label
            if np.sum(I2)>th:
                R = R | I2
    return np.float32(255*R)

In [27]:
# Function to clamp bounding box coordinates
def clampBox(bbox, img_shape):
    x, y, w, h = bbox
    x = max(0, min(x, img_shape[1] - 1))
    y = max(0, min(y, img_shape[0] - 1))
    w = max(0, min(w, img_shape[1] - x))
    h = max(0, min(h, img_shape[0] - y))
    return (x, y, w, h)

In [55]:
# Saves sequence into folder in 'out'
def saveSeq(C,counter,th,trackingFrames, trackingBboxScaleFactor, outputPath):
    if len(C)<th:
        return

    firstFrameOfClip = counter - len(C)

    totalDetectionFrames, totalTrackingFrames = 0, 0
    trackers = None
    consecutivePersonFrames = 1
    k = 0
    trackingPerson = False
    detection = False
    for frame in C:

        # Searches for the next frame with at least one person present
        if not trackingPerson or consecutivePersonFrames > trackingFrames:
            # Detects objects
            bboxs,labels,conf = cv.detect_common_objects(frame, confidence= 0.4, model='yolov4-tiny')
            
            # Checks if at least one person is present
            person_indices = [i for i, label in enumerate(labels) if label == 'person']
            if person_indices:
                totalDetectionFrames += 1
                trackingPerson = True
                detection = True
                consecutivePersonFrames = 1 
                if len(labels) > 1:
                    # Use the indices to filter bboxs, labels, and conf
                    bboxs = [bboxs[i] for i in person_indices]
                    labels = [labels[i] for i in person_indices]
                    conf = [conf[i] for i in person_indices]
                  
                # Initializes multi-object tracker
                trackers = cv2.legacy.MultiTracker_create()
                for box in bboxs:
                    tracker = cv2.legacy.TrackerMOSSE_create()
                    box = clampBox(box, frame.shape) # Ensures the bbox is within the image

                    trackers.add(tracker, frame, box)
            else:
                trackingPerson = False
                continue # Ignores frames without people
        else:
            # Updates tracking for each object
            totalTrackingFrames += 1
            success, newBoxes = trackers.update(frame)
            bboxs = []
            labels = []  
            conf = []
            for newBox in newBoxes:
                # Convert (x, y, w, h) to (xmin, ymin, xmax, ymax)
                x, y, w, h = map(int, newBox)
                bbox = (x, y, x + int(w*trackingBboxScaleFactor), y + int(h*trackingBboxScaleFactor)) 
                bboxs.append(bbox)
                labels.append('person') 
                conf.append(1.0)  # Adds a placeholder confidence score
            consecutivePersonFrames += 1

        k += 1
        imName = str(firstFrameOfClip)+'_'+ str(k)+'.jpg'
        finalPath = os.path.join(outputPath, imName)
        frame = draw_bbox(frame,bboxs,labels,conf, write_conf=detection)
        cv2.imwrite(finalPath,frame)
        detection = False

    print(totalDetectionFrames, totalTrackingFrames, len(C))
        

In [28]:
def displaySeq(outPutPath):
    for imName in os.listdir(outPutPath):
        frame = cv2.imread(os.path.join(outPutPath,imName))
        frame = cv2.resize(frame,dsize=(600,400))
        cv2.imshow('Display',frame)
        k = cv2.waitKey(30) & 0xff
        if k == 27:
            break
    cv2.destroyAllWindows()

# Frame-by-frame capture:

In [172]:
# Checks for valid input/output paths
try:
    if not output_folder.value or not input_path.value:
        raise ValueError("Output folder and input path cannot be empty or whitespace")

    outputPath = os.path.join(os.getcwd(), 'out', output_folder.value.strip())
    inputPath = os.path.join(os.getcwd(), 'src', input_path.value.strip())

    os.makedirs(outputPath, exist_ok=True)
    
    if not os.path.exists(inputPath):
        raise FileNotFoundError(f"{inputPath} does not exist")

except (OSError, ValueError) as e:
    print(f"Error: {e}")
    raise


imPath = f'src/{input_path.value}'
fgModel = cv2.createBackgroundSubtractorMOG2()

idx = []
C = []
counter = 0

for imName in os.listdir(imPath):
    counter += 1

    frame = cv2.imread(os.path.join(imPath,imName))
    frame = cv2.resize(frame,dsize=(600,400))

    # Denoises frame
    fgmask = fgModel.apply(frame)
    K_r = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    fgmask = cv2.morphologyEx(np.float32(fgmask),cv2.MORPH_OPEN,K_r)

    # Keeps only large connected components
    num_labels,labels_im = cv2.connectedComponents(np.array(fgmask>0,np.uint8))
    fgmask = keepLargeComponents(labels_im,minComponentSize.value) 

    if np.sum(fgmask)>0:
        idx.append(counter)
        C.append(frame)

    # Saves clip during continous changes longer than 2 frames and at least 1 frame away from the last clip
    if len(idx) > 2 and idx[-1] > idx[-2]+1:
        saveSeq(C[:-1],counter,minClipLength.value, trackingFrames.value, trackingBboxScaleFactor.value, outputPath)
        idx = []
        C = []
    
    # Displays original frame alongside masked frame
    F = np.zeros(frame.shape,np.uint8)
    F[:,:,0],F[:,:,1],F[:,:,2] = fgmask, fgmask,fgmask
    F2 = np.hstack((frame,F))
    cv2.imshow('Display',F2)

    # Break if 'esc' key is pressed
    k = cv2.waitKey(5) & 0xff
    if k == 27:
        break
saveSeq(C,counter,minClipLength.value, trackingFrames.value, trackingBboxScaleFactor.value, outputPath)
cv2.destroyAllWindows()

# On normalFootage (else held standard):

# 30 tf, 10.9s
# 20 tf, 12.2s
# 10 tf, 12.5s
# 5 tf, 12.9s
# 1 tf, 14.9s
# 0 tf, 18.0s

# On noisyFootage (150 clip, 600 minCompSize):

# 25 tf, 1m, 11.7s
# 10 tf, 1m 13.9s
# 3 tf, 1m 17.6
# 0 tf, 1m 24.4s

KeyboardInterrupt: 

# Video Capture:

In [9]:
vidPath = r'src\busyStreet.mp4'
video = cv2.VideoCapture(vidPath)

fgModel = cv2.createBackgroundSubtractorMOG2()

leastNumOfFrames = 5
idx = []
C = []
counter = 0

while(True):
    ret, frame = video.read()
    if ret == False:
        break
    counter += 1

    frame = cv2.resize(frame,dsize=(600,400))

    # Denoises frame
    fgmask = fgModel.apply(frame)
    K_r = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    fgmask = cv2.morphologyEx(np.float32(fgmask),cv2.MORPH_OPEN,K_r)

    num_labels,labels_im = cv2.connectedComponents(np.array(fgmask>0,np.uint8))
    fgmask = keepLargeComponents(labels_im,100) #parameterize in ui

    if np.sum(fgmask)>0:
        idx.append(counter)
        C.append(frame)

    # Saves frame during continous changes longer than 2 frames
    if len(idx) > 2 and idx[-1] > idx[-2]+1:
        saveSeq(C,counter,leastNumOfFrames,outPutPath)
        idx = []
        C = []

    F = np.zeros(frame.shape,np.uint8)
    F[:,:,0],F[:,:,1],F[:,:,2] = fgmask, fgmask,fgmask
    F2 = np.hstack((frame,F))
    cv2.imshow('Display',F2)

    # Break if 'esc' is pressed
    k = cv2.waitKey(5) & 0xff
    if k == 27:
        break
saveSeq(C,counter,leastNumOfFrames,outPutPath)
video.release()
cv2.destroyAllWindows()
    

NameError: name 'outPutPath' is not defined